In [1]:
import os
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/rahulsamantcoc2/End-to-End_Flash_App.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="rahulsamantcoc2"
os.environ['MLFLOW_TRACKING_PASSWORD']="33607bcb15d4e7a7cca29f0f443d16762cc15549"

In [2]:
%pwd

'd:\\End-to-End_Flash_App\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\End-to-End_Flash_App'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvalConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_eval_config(self) -> ModelEvalConfig:
        config=self.config.model_evaluation
        params=self.params.gini
        schema=self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        model_eval_config=ModelEvalConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/rahulsamantcoc2/End-to-End_Flash_App.mlflow"
        )
        return model_eval_config

In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [15]:
class ModelEval:
    def __init__(self, config: ModelEvalConfig):
        self.config = config
    def Eval(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.Eval(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="DecisionTreeModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [16]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_eval_config()
    model_eval = ModelEval(config=model_eval_config)
    model_eval.log_into_mlflow()
except Exception as e:
    raise e

[2024-12-19 05:47:48,009: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-19 05:47:48,015: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-19 05:47:48,015: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-19 05:47:48,021: INFO: common: created directory at: artifacts]
[2024-12-19 05:47:48,028: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-19 05:47:48,511: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/12/19 05:48:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'DecisionTreeModel'.
2024/12/19 05:48:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DecisionTreeModel, version 1
Created version '1' of model 'DecisionTreeModel'.


🏃 View run delightful-mule-811 at: https://dagshub.com/rahulsamantcoc2/End-to-End_Flash_App.mlflow/#/experiments/0/runs/6646099e77e24b1d93444cc0b0ca06a1
🧪 View experiment at: https://dagshub.com/rahulsamantcoc2/End-to-End_Flash_App.mlflow/#/experiments/0
